In [ ]:
import math
import time
import pinocchio as pin
import numpy as np
from numpy.random import rand
from numpy.linalg import inv,pinv,norm,eig,svd
import matplotlib.pylab as plt; plt.ion()
import quadprog
from tp6.robot_hand import RobotHand
from tp6.meshcat_viewer_wrapper import MeshcatVisualizer
from tp6.traj_ref import TrajRef
from tp6.collision_wrapper import CollisionWrapper

In [ ]:
robot=RobotHand()
viz = MeshcatVisualizer(robot)
viz.display(robot.q0)
viz.viewer.jupyter_cell()

In [ ]:
qdes = TrajRef(robot.q0,omega = np.array([0,.1,1,1.5,2.5,-1,-1.5,-2.5,.1,.2,.3,.4,.5,.6]),amplitude=1.5)
colwrap = CollisionWrapper(robot)

q = robot.q0.copy()
vq = np.zeros(robot.model.nv)
dt = 1e-3
Kp = 50
Kv = 2 * np.sqrt(Kp)
hq = []
hqdes = []
previousContacts = []
for it in range(10000):
    t = it*dt
    ### Compute M, b, tauq and self-acceleration (q0)
    M = pin.crba(robot.model,robot.data,q)
    b = pin.rnea(robot.model,robot.data,q,vq,np.zeros(robot.model.nv))
    tauq = -Kp*(q - qdes(t)) - Kv*(vq-qdes.velocity(t))
    aq = inv(M) @ (tauq - b)
    
    #get the collisions
    colwrap.computeCollisions(q)
    collisions = colwrap.getCollisionList()
    if (len(collisions) > 0): # there is a collision !
        # get the distances and select the ones with distances <= 1e-3
        dist=colwrap.getCollisionDistances(collisions)
        myCollisions = []
        isNewCollision = False
        for i in range(len(collisions)):
            if (dist[i] < 1e-3):
                myCollisions.append(collisions[i])
                isNewCollision = isNewCollision or (not(collisions[i] in previousContacts))
                
        #compute the Jacobians
        J = colwrap.getCollisionJacobian(myCollisions)
        dist=colwrap.getCollisionDistances(myCollisions)
        
        #if there is new collisions, nullify the velocities
        if(isNewCollision):
            vq = vq - pinv(J)@J@vq
            
        #compute the acceleration thanks to quadprod
        colwrap.computeCollisions(q,vq)
        coriollis = colwrap.getCollisionJdotQdot(myCollisions)
        
        d = -Kp*dist-Kv*J@vq-coriollis
        b = np.zeros(robot.model.nv)
        # a = ddotq - ddotq0
        [a,cost,_,niter,lag,iact] = quadprog.solve_qp(M,b,J.T,d[0])
        aq += a
        
    vq += aq*dt
    q = pin.integrate(robot.model,q,vq*dt)
    hq.append(q.copy())
    hqdes.append(qdes(t).copy())

    if it%20==0: 
        viz.display(q)
        time.sleep(dt)
                
    
    
    

In [ ]:
plt.subplot(211)
plt.plot([q[2] for q in hq])
plt.plot([q[2] for q in hqdes])
plt.ylabel('Joint #2')
plt.subplot(212)
plt.plot([q[3] for q in hq])
plt.plot([q[3] for q in hqdes]);
plt.ylabel('Joint #3')
plt.xlabel('Iterations')